In [1]:
from model import *
m = Model()

In [6]:
def chunk():
    loss = None
    for i in range(50):
        for j in range(1000):
            loss = m.train_batch(100)
        print('{}/ loss = {}'.format(i, loss))

In [8]:
chunk()

0/ loss = 0.5567288994789124
1/ loss = 0.5669138431549072
2/ loss = 0.5202236175537109
3/ loss = 0.5442884564399719
4/ loss = 0.5264337062835693
5/ loss = 0.5120351910591125
6/ loss = 0.49883487820625305
7/ loss = 0.480994313955307
8/ loss = 0.46732425689697266
9/ loss = 0.446341335773468
10/ loss = 0.45113763213157654
11/ loss = 0.4204480051994324
12/ loss = 0.4136160612106323
13/ loss = 0.3862909972667694
14/ loss = 0.40032798051834106
15/ loss = 0.3811221420764923
16/ loss = 0.365391343832016
17/ loss = 0.37656861543655396
18/ loss = 0.35959967970848083
19/ loss = 0.36451515555381775
20/ loss = 0.3561052083969116
21/ loss = 0.36729854345321655
22/ loss = 0.35288169980049133
23/ loss = 0.3548228442668915
24/ loss = 0.35952994227409363
25/ loss = 0.3482111692428589
26/ loss = 0.3550739288330078
27/ loss = 0.3550228774547577
28/ loss = 0.3516204059123993
29/ loss = 0.3504962623119354
30/ loss = 0.3514910936355591
31/ loss = 0.35069766640663147
32/ loss = 0.3620469570159912
33/ loss = 0

KeyboardInterrupt: 

In [9]:
m.score()

3 / 1000 = 0.3%


In [11]:
m.sample()

input puzzle: 11000*11000    
right answer: 10001010000   
wrong answer: 100100100000  

input puzzle: 100111*100111  
right answer: 10110100011   
wrong answer: 10001110011   

input puzzle: 1010101*1010101
right answer: 10100000101101
wrong answer: 10000110101101

input puzzle: 1101110*1101110
right answer: 10010111010   
wrong answer: 101000110100  

input puzzle: 101100*101100  
right answer: 1001111110000 
wrong answer: 1000101110000 

input puzzle: 110010*110010  
right answer: 11101101100   
wrong answer: 100001011000  

input puzzle: 1000*1000      
right answer: 1100001000    
wrong answer: 1100000000    

input puzzle: 110011*110011  
right answer: 1000010111100 
wrong answer: 1011110111100 

input puzzle: 1001101*1001101
right answer: 1101101011111 
wrong answer: 1000010011111 

input puzzle: 10010*10010    
right answer: 100000101000  
wrong answer: 10000110100   

